<a href="https://colab.research.google.com/github/nibaskumar93n-debug/Morphoinformatics/blob/main/Otu_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!mkdir -p raw_fastq2
!cd raw_fastq2 && bash /content/ena-file-download-selected-files-20260107-0904.sh

File ‘ERR12102849_2.fastq.gz’ already there; not retrieving.
File ‘ERR12102855_2.fastq.gz’ already there; not retrieving.
File ‘ERR12102867_2.fastq.gz’ already there; not retrieving.
File ‘ERR12102882_2.fastq.gz’ already there; not retrieving.
File ‘ERR12102873_2.fastq.gz’ already there; not retrieving.
File ‘ERR12102861_2.fastq.gz’ already there; not retrieving.
File ‘ERR12102894_2.fastq.gz’ already there; not retrieving.
File ‘ERR12102997_1.fastq.gz’ already there; not retrieving.
File ‘ERR12103015_1.fastq.gz’ already there; not retrieving.
File ‘ERR12102994_1.fastq.gz’ already there; not retrieving.
File ‘ERR12103036_1.fastq.gz’ already there; not retrieving.
File ‘ERR12103009_1.fastq.gz’ already there; not retrieving.
File ‘ERR12103030_1.fastq.gz’ already there; not retrieving.
File ‘ERR12103027_1.fastq.gz’ already there; not retrieving.
File ‘ERR12102897_2.fastq.gz’ already there; not retrieving.
File ‘ERR12102982_1.fastq.gz’ already there; not retrieving.
File ‘ERR12102931_1.fast

In [ ]:
# Count all files
!ls raw_fastq2 | wc -l

# Or directly
!find raw_fastq2 -type f | wc -l

210
210


In [ ]:
# Install Java (required for FastQC)
!apt-get update
!apt-get install -y default-jre

# Download and install FastQC
!wget https://www.bioinformatics.babraham.ac.uk/projects/fastqc/fastqc_v0.12.1.zip
!unzip fastqc_v0.12.1.zip
!chmod +x FastQC/fastqc

# Add to PATH
import os
os.environ['PATH'] += ':/content/FastQC'

In [ ]:
!mkdir -p fastqc_results2
!fastqc /content/raw_fastq2/*.fastq* -o fastqc_results2

In [ ]:
# Zip the folder
!zip -r fastqc_results2.zip fastqc_results2/

In [ ]:
!pip install multiqc
!multiqc --version

In [ ]:
!multiqc fastqc_results2 -o multiqc_report2
!ls -lh multiqc_report2

In [ ]:
# Zip the folder
!zip -r multiqc_report2.zip multiqc_report2/

In [ ]:
## Trimomatic install
!apt-get update -qq
!apt-get install -y openjdk-11-jre-headless
!wget -q https://github.com/usadellab/Trimmomatic/releases/download/v0.39/trimmomatic-0.39.zip
!unzip -q trimmomatic-0.39.zip
!java -jar Trimmomatic-0.39/trimmomatic-0.39.jar -version

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Suggested packages:
  libnss-mdns fonts-dejavu-extra fonts-ipafont-gothic fonts-ipafont-mincho
  fonts-wqy-microhei | fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-11-jre-headless
0 upgraded, 1 newly installed, 0 to remove and 25 not upgraded.
Need to get 42.6 MB of archives.
After this operation, 176 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 openjdk-11-jre-headless amd64 11.0.29+7-1ubuntu1~22.04 [42.6 MB]
Fetched 42.6 MB in 1s (31.0 MB/s)
Selecting previously unselected package openjdk-11-jre-headless:amd64.
(Reading database ... 117528 files and directories currently installed.)
Preparing to unpack ...

In [ ]:
import os
import glob
from tqdm import tqdm
import subprocess

# ========================================
# Configuration
# ========================================
INPUT_DIR = "/content/raw_fastq2"
OUTPUT_DIR = "/content/trimmed_fastq"
ADAPTERS = "Trimmomatic-0.39/adapters/TruSeq3-PE.fa"
THREADS = 4
LOG_DIR = "/content/trimmomatic_logs"

# Create output directories
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(LOG_DIR, exist_ok=True)

# ========================================
# Get all forward read files
# ========================================
forward_files = sorted(glob.glob(f"{INPUT_DIR}/*_1.fastq.gz"))
print(f"Found {len(forward_files)} forward read files")

# ========================================
# Process each sample (keeping only paired)
# ========================================
failed_samples = []

for forward_file in tqdm(forward_files, desc="Processing samples"):
    # Get sample name
    sample_name = os.path.basename(forward_file).replace('_1.fastq.gz', '')
    reverse_file = forward_file.replace('_1.fastq.gz', '_2.fastq.gz')

    # Check if reverse file exists
    if not os.path.exists(reverse_file):
        print(f"⚠️  Warning: Reverse file not found for {sample_name}")
        failed_samples.append(sample_name)
        continue

    # Define output files (ONLY PAIRED)
    paired1 = f"{OUTPUT_DIR}/{sample_name}_1_paired.fq.gz"
    paired2 = f"{OUTPUT_DIR}/{sample_name}_2_paired.fq.gz"
    log_file = f"{LOG_DIR}/{sample_name}_trimmomatic.log"

    # Skip if already processed
    if os.path.exists(paired1) and os.path.exists(paired2):
        print(f"✓ Skipping {sample_name} (already processed)")
        continue

    # Run Trimmomatic - send unpaired to /dev/null
    cmd = [
        "java", "-jar", "Trimmomatic-0.39/trimmomatic-0.39.jar", "PE",
        "-threads", str(THREADS),
        forward_file, reverse_file,
        paired1, "/dev/null",      # unpaired1 → discarded
        paired2, "/dev/null",      # unpaired2 → discarded
        f"ILLUMINACLIP:{ADAPTERS}:2:30:10",
        "LEADING:3", "TRAILING:3", "SLIDINGWINDOW:4:15", "MINLEN:36"
    ]

    try:
        with open(log_file, 'w') as log:
            result = subprocess.run(cmd, stdout=log, stderr=log, check=True)
        print(f"✓ Completed: {sample_name}")
    except subprocess.CalledProcessError as e:
        print(f"❌ Failed: {sample_name}")
        failed_samples.append(sample_name)

# ========================================
# Summary
# ========================================
print("\n" + "="*50)
print("Processing Complete!")
print("="*50)
print(f"Total samples: {len(forward_files)}")
print(f"Successful: {len(forward_files) - len(failed_samples)}")
print(f"Failed: {len(failed_samples)}")

if failed_samples:
    print("\n⚠️  Failed samples:")
    for sample in failed_samples:
        print(f"  - {sample}")

# Show output files
print(f"\n📁 Paired output files saved in: {OUTPUT_DIR}")
print(f"   Pattern: *_1_paired.fq.gz and *_2_paired.fq.gz")

Found 105 forward read files


Processing samples:   1%|          | 1/105 [00:08<14:54,  8.60s/it]

✓ Completed: ERR12102845


Processing samples:   2%|▏         | 2/105 [00:14<12:23,  7.22s/it]

✓ Completed: ERR12102847


...

✓ Completed: ERR12103036


Processing samples:  99%|█████████▉| 104/105 [19:15<00:11, 11.96s/it]

✓ Completed: ERR12103038


Processing samples: 100%|██████████| 105/105 [19:26<00:00, 11.11s/it]

✓ Completed: ERR12103040

Processing Complete!
Total samples: 105
Successful: 105
Failed: 0

📁 Paired output files saved in: /content/trimmed_fastq
   Pattern: *_1_paired.fq.gz and *_2_paired.fq.gz


In [ ]:
!apt-get install -y build-essential zlib1g-dev

# Clone NGmerge repository
!git clone https://github.com/jsh58/NGmerge.git

# Compile NGmerge
!cd NGmerge && make

# Move binary to PATH
!cp NGmerge/NGmerge /usr/local/bin/

# Verify installation
!NGmerge -h

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
zlib1g-dev is already the newest version (1:1.2.11.dfsg-2ubuntu9.2).
zlib1g-dev set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.
Cloning into 'NGmerge'...
remote: Enumerating objects: 176, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 176 (delta 32), reused 36 (delta 16), pack-reused 123 (from 1)
Receiving objects: 100% (176/176), 864.64 KiB | 20.59 MiB/s, done.
Resolving deltas: 100% (95/95), done.
gcc -g -Wall -std=gnu99 -fopenmp -O2 -o NGmerge NGmerge.c -lz
Usage: ./NGmerge {-1 <file> -2 <file> -o <file>}  [optional arguments]
Required arguments:
  -1  <file>       Input FASTQ file with reads from forward direction
  -2  <file>       Input FASTQ file with reads from reverse direction
  -o  <file>       Output FASTQ fil

In [ ]:
!NGmerge --version

NGmerge, version 0.5
Copyright (C) 2017 John M. Gaspar (jsh58@wildcats.unh.edu)


In [ ]:
import os
import glob
import subprocess
import re
from tqdm import tqdm

# Paths
INPUT_DIR = "/content/trimmed_fastq"
OUTPUT_DIR = "/content/merged_reads"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Get all forward reads
forward_files = glob.glob(f"{INPUT_DIR}/*_1_paired.fq.gz")
print(f"Found {len(forward_files)} samples to merge\n")

# Track statistics
total_reads = 0
total_merged = 0
results = []

# Merge each sample
for fwd in tqdm(forward_files, desc="Merging reads"):
    sample = os.path.basename(fwd).replace('_1_paired.fq.gz', '')
    rev = fwd.replace('_1_paired.fq.gz', '_2_paired.fq.gz')
    output = f"{OUTPUT_DIR}/{sample}_merged"

    # NGmerge command
    cmd = f"NGmerge -1 {fwd} -2 {rev} -o {output} -z -n 4 -v"
    result = subprocess.run(cmd, shell=True, capture_output=True, text=True)

    # Parse output for statistics
    output_text = result.stderr + result.stdout

    reads_analyzed = 0
    reads_merged = 0

    if match := re.search(r'Reads analyzed:\s+(\d+)', output_text):
        reads_analyzed = int(match.group(1))
    if match := re.search(r'Successfully stitched:\s+(\d+)', output_text):
        reads_merged = int(match.group(1))

    pct = (reads_merged / reads_analyzed * 100) if reads_analyzed > 0 else 0

    total_reads += reads_analyzed
    total_merged += reads_merged
    results.append((sample, reads_analyzed, reads_merged, pct))

# Summary
print("\n" + "="*70)
print("📊 Merge Statistics")
print("="*70)

# Show first 10 samples
print("\nFirst 10 samples:")
for sample, analyzed, merged, pct in results[:10]:
    print(f"  {sample:25s}: {merged:>8,} / {analyzed:>8,} ({pct:5.1f}%)")

if len(results) > 10:
    print(f"  ... and {len(results)-10} more samples")

# Overall statistics
overall_pct = (total_merged / total_reads * 100) if total_reads > 0 else 0
print("\n" + "="*70)
print(f"📈 Overall Results:")
print(f"   Total reads:   {total_reads:>12,}")
print(f"   Merged:        {total_merged:>12,} ({overall_pct:.1f}%)")
print(f"   Not merged:    {total_reads - total_merged:>12,} ({100-overall_pct:.1f}%)")
print("="*70)

# Check output files
merged_files = len(glob.glob(f'{OUTPUT_DIR}/*.fastq.gz'))
print(f"\n✅ Done! Created {merged_files} merged files in: {OUTPUT_DIR}")

Found 105 samples to merge



Merging reads: 100%|██████████| 105/105 [08:16<00:00,  4.73s/it]


📊 Merge Statistics

First 10 samples:
  ERR12103017              :    1,230 /        0 (  0.0%)
  ERR12102946              :    1,876 /        0 (  0.0%)
  ERR12103028              :      290 /        0 (  0.0%)
  ERR12102882              :       86 /        0 (  0.0%)
  ERR12102992              :    2,579 /        0 (  0.0%)
  ERR12102999              :       29 /        0 (  0.0%)
  ERR12102958              :    2,096 /        0 (  0.0%)
  ERR12102896              :   23,269 /        0 (  0.0%)
  ERR12102968              :    2,154 /        0 (  0.0%)
  ERR12102851              :       48 /        0 (  0.0%)
  ... and 95 more samples

📈 Overall Results:
   Total reads:              0
   Merged:             336,061 (0.0%)
   Not merged:        -336,061 (100.0%)

✅ Done! Created 0 merged files in: /content/merged_reads


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
# Install Miniconda
!wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local

# Initialize conda
import sys
sys.path.append('/usr/local/lib/python3.10/site-packages')

# Accept conda Terms of Service
!conda config --set channel_priority flexible
!conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/main
!conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/r

# Update conda
!conda update -n base -c defaults conda -y

--2026-01-11 05:32:52--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.191.158, 104.16.32.241, 2606:4700::6810:bf9e, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.191.158|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 156772981 (150M) [application/octet-stream]
Saving to: ‘Miniconda3-latest-Linux-x86_64.sh’

Miniconda3-latest-L 100%[===================>] 149.51M   163MB/s    in 0.9s    

2026-01-11 05:32:53 (163 MB/s) - ‘Miniconda3-latest-Linux-x86_64.sh’ saved [156772981/156772981]

PREFIX=/usr/local
Unpacking bootstrapper...
Unpacking payload...

Installing base environment...

Preparing transaction: ...working... done
Executing transaction: ...working... done
installation finished.
    You currently have a PYTHONPATH environment variable set. This may cause
    unexpected behavior when running the Python interpreter in Miniconda3.
    For best results, please

In [2]:
# Install Miniconda (already done from previous step, skip if already installed)
# !wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
# !chmod +x Miniconda3-latest-Linux-x86_64.sh
# !bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local

# Accept Terms of Service (already done)
# !conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/main
# !conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/r


# Create QIIME2 environment with the correct URL (2024.10 version with py310)
!conda env create -n qiime2-amplicon-2024.10 --file https://data.qiime2.org/distro/amplicon/qiime2-amplicon-2024.10-py310-linux-conda.yml

print("✓ QIIME2 installation complete!")

Streaming output truncated to the last 5000 lines.























































































































































































































































































































































































































































































































































































































































































































































































































































































































































































In [3]:
import zipfile

with zipfile.ZipFile('Merged.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/DKD_data')
print("Extraction complete!")

Extraction complete!


In [6]:
import pandas as pd
import os

# Define the path to your merged files
data_dir = "/content/DKD_data/Merged"

# Get all merged fastq files
fastq_files = [f for f in os.listdir(data_dir) if f.endswith('.fastq') or f.endswith('.fastq.gz')]

# Create manifest dataframe
manifest_data = []
for file in sorted(fastq_files):
    # Extract sample ID from filename (remove _merged.fastq or similar)
    sample_id = file.replace('_merged.fastq.gz', '').replace('_merged.fastq', '').replace('.fastq.gz', '').replace('.fastq', '')

    # Full absolute path
    filepath = os.path.join(data_dir, file)

    manifest_data.append({'sample-id': sample_id, 'absolute-filepath': filepath})

# Create dataframe
manifest_df = pd.DataFrame(manifest_data)

# Save manifest file
manifest_df.to_csv('manifest_gut.tsv', sep='\t', index=False)

print("Manifest file created!")
print(f"Total samples: {len(manifest_df)}")
print("\nFirst few rows:")
print(manifest_df.head())

Manifest file created!
Total samples: 319

First few rows:
     sample-id                                  absolute-filepath
0  SRR22270903  /content/DKD_data/Merged/SRR22270903_merged.fa...
1  SRR22270904  /content/DKD_data/Merged/SRR22270904_merged.fa...
2  SRR22270905  /content/DKD_data/Merged/SRR22270905_merged.fa...
3  SRR22270906  /content/DKD_data/Merged/SRR22270906_merged.fa...
4  SRR22270907  /content/DKD_data/Merged/SRR22270907_merged.fa...


In [7]:
# Import already-merged/single-end sequences from manifest file
!source /usr/local/etc/profile.d/conda.sh && conda activate qiime2-amplicon-2024.10 && qiime tools import \
  --input-path manifest.tsv \
  --output-path demux.qza \
  --type 'SampleData[JoinedSequencesWithQuality]' \
  --input-format SingleEndFastqManifestPhred33V2

Imported manifest.tsv as SingleEndFastqManifestPhred33V2 to demux.qza


In [8]:
### qiime file validation
!source /usr/local/etc/profile.d/conda.sh && conda activate qiime2-amplicon-2024.10 && qiime tools validate demux.qza

Result demux.qza appears to be valid at level=max.


In [9]:
# Dereplicate sequences (identify unique sequences and their abundances)
!source /usr/local/etc/profile.d/conda.sh && conda activate qiime2-amplicon-2024.10 && qiime vsearch dereplicate-sequences \
  --i-sequences demux.qza \
  --o-dereplicated-table table-derep.qza \
  --o-dereplicated-sequences rep-seqs-derep.qza

Saved FeatureTable[Frequency] to: table-derep.qza
Saved FeatureData[Sequence] to: rep-seqs-derep.qza


In [11]:
!source /usr/local/etc/profile.d/conda.sh && conda activate qiime2-amplicon-2024.10 && \
qiime tools import \
  --type 'FeatureData[Sequence]' \
  --input-path 97_otus.fasta \
  --output-path 97_otus.qza

Imported 97_otus.fasta as DNASequencesDirectoryFormat to 97_otus.qza


In [ ]:
!source /usr/local/etc/profile.d/conda.sh && conda activate qiime2-amplicon-2024.10 && \
qiime vsearch cluster-features-open-reference \
--i-table table-derep.qza \
--i-sequences rep-seqs-derep.qza \
--i-reference-sequences 97_otus.qza \
--p-perc-identity 0.97 \
--o-clustered-table table-97.qza \
--o-clustered-sequences rep-seqs-97.qza \
--o-new-reference-sequences new-ref-seqs-97.qza